In [10]:
import pandas as pd
import numpy as np  

In [ ]:

# Path to your Excel file
excel_path = r'C:\Users\yupen\Documents\stash\data1.xlsx'  # Use raw string to avoid unicode escape error
# Read all sheets into a dictionary of DataFrames
all_sheets = pd.read_excel(excel_path, sheet_name=None)
# Merge all sheets into one DataFrame, adding a column for sheet name
merged_df1 = pd.concat([df.assign(sheet_name=sheet) for sheet, df in all_sheets.items()], ignore_index=True)

# Path to your Excel file
excel_path = r'C:\Users\yupen\Documents\stash\data2.xlsx'  # Use raw string to avoid unicode escape error
# Read all sheets into a dictionary of DataFrames
all_sheets = pd.read_excel(excel_path, sheet_name=None)
# Merge all sheets into one DataFrame, adding a column for sheet name
merged_df2 = pd.concat([df.assign(sheet_name=sheet) for sheet, df in all_sheets.items()], ignore_index=True)

df = pd.concat([merged_df1, merged_df2], ignore_index=True)

In [17]:
# Create a mapping dictionary between sheet names and sym values
sheet_to_sym = {
    'audusd': 'AUDUSD', 'c 1': 'C1', 'cl1': 'CL1', 'co1': 'CO1', 'eurusd': 'EURUSD', 'gbpusd': 'GBPUSD', 
    'gjgb1': 'GJGB1', 'gjgb10': 'GJGB10', 'gjgb30': 'GJGB30', 'gjgb5': 'GJGB5', 'hg1': 'HG1', 'hsi': 'HSI',
    'hxc': 'HXC', 'ihn+1m': 'USDIDR', 'irn+1m': 'USDINR', 'kc1': 'KC1', 'kwn+1m': 'USDKRW', 'la1': 'LA1', 
    'ndx': 'NDX', 'ng1': 'NG1', 'nky': 'NKY', 'ntn+1m': 'USDTWD', 'nzdusd': 'NZDUSD', 'ppn+1m': 'USDPHP', 
    'rr1': 'RR1', 'shcomp': 'SHCOMP', 'shsz300': 'SHSZ300', 'spx': 'SPX', 'ukx': 'UKX', 'usdcad': 'USDCAD', 
    'usdchf': 'USDCHF', 'usdcnh': 'USDCNH', 'usddxy': 'USDDXY', 'usdjpy': 'USDJPY', 'usdnok': 'USDNOK',
    'usdsek': 'USDSEK', 'usdsgd': 'USDSGD', 'usdthb': 'USDTHB', 'usgg10yr': 'US10', 'usgg12m': 'US1', 
    'usgg1m': 'US1M', 'usgg30yr': 'US30', 'usgg5yr': 'US5', 'w 1': 'W1', 'xag': 'XAGUSD', 
    'xau': 'XAUUSD', 'xbt': 'BTCUSD', 'xetusd': 'ETHUSD'
}

# Create a new column 'sym' using the mapping
df['sym'] = df['sheet_name'].map(sheet_to_sym)

In [38]:
asset_name = [
    'GJGB10', 'HSI', 'HXC', 'NDX', 'SPX', 'NKY', 'SHCOMP', 'SHSZ300', 'UKX', 'USDDXY', 
    'US10', 'US1M', 'US30', 'XAGUSD', 'XAUUSD'
 ]

# Select only relevant columns and rows for pivoting
df_selected = df[df['sym'].isin(asset_name)][['Date', 'sym', 'Close']]

# Pivot the DataFrame: index='Date', columns='sym', values='Close'
pivot_df = df_selected.pivot(index='Date', columns='sym', values='Close')

# Display the pivoted DataFrame
pivot_df.head()

sym,GJGB10,HSI,HXC,NDX,NKY,SHCOMP,SHSZ300,SPX,UKX,US10,US1M,US30,USDDXY,XAGUSD,XAUUSD
Date,,,,,,,,,,,,,,,
2000-01-03,NaN,17369.63,NaN,3790.55,NaN,NaN,NaN,1455.22,NaN,6.592,NaN,6.622,100.22,5.405,289.00
2000-01-04,1.738,17072.82,NaN,3546.20,19002.86,1406.371,NaN,1399.42,6665.9,6.497,NaN,6.537,100.41,5.325,282.25
2000-01-05,1.699,15846.72,NaN,3507.31,18542.55,1409.682,NaN,1402.11,6535.9,6.594,NaN,6.621,100.38,5.145,280.00
2000-01-06,1.685,15153.23,NaN,3340.81,18168.27,1463.942,NaN,1403.45,6447.2,6.524,NaN,6.553,100.48,5.120,281.10
2000-01-07,1.719,15405.63,NaN,3529.60,18193.41,1516.604,NaN,1441.47,6504.8,6.515,NaN,6.546,100.72,5.175,281.75


In [41]:
# Forward fill missing values and drop any remaining nulls
pivot_df_filled = pivot_df.ffill().dropna()
# Reset index so the key column (Date) becomes a normal column
pivot_df_final = pivot_df_filled.reset_index()

# Display the final DataFrame
pivot_df_final.head()

sym,Date,GJGB10,HSI,HXC,NDX,NKY,SHCOMP,SHSZ300,SPX,UKX,US10,US1M,US30,USDDXY,XAGUSD,XAUUSD
0,2002-01-04,1.403,11702.15,1351.74,1675.03,10871.49,1611.393,1316.46,1172.51,5323.8,5.128,1.710,5.548,116.33,4.70,278.95
1,2002-01-07,1.389,11892.64,1375.23,1649.83,10942.36,1596.760,1302.08,1164.89,5293.6,5.049,1.690,5.488,116.33,4.71,278.75
2,2002-01-08,1.399,11713.71,1394.96,1666.58,10695.60,1583.461,1292.71,1160.71,5250.4,5.080,1.700,5.529,116.83,4.69,279.05
3,2002-01-09,1.385,11440.72,1357.56,1653.76,10663.98,1561.349,1272.65,1155.14,5228.5,5.051,1.629,5.497,117.01,4.82,283.95
4,2002-01-10,1.395,11256.07,1329.60,1656.68,10538.43,1576.437,1281.26,1156.55,5190.7,4.979,1.670,5.416,116.88,4.79,286.35


sym,Date,GJGB10,HSI,HXC,NDX,NKY,SHCOMP,SHSZ300,SPX,UKX,US10,US1M,US30,USDDXY,XAGUSD,XAUUSD
0,2002-01-04,1.403,11702.15,1351.74,1675.03,10871.49,1611.393,1316.46,1172.51,5323.8,5.128,1.710,5.548,116.33,4.70,278.95
1,2002-01-07,1.389,11892.64,1375.23,1649.83,10942.36,1596.760,1302.08,1164.89,5293.6,5.049,1.690,5.488,116.33,4.71,278.75
2,2002-01-08,1.399,11713.71,1394.96,1666.58,10695.60,1583.461,1292.71,1160.71,5250.4,5.080,1.700,5.529,116.83,4.69,279.05
3,2002-01-09,1.385,11440.72,1357.56,1653.76,10663.98,1561.349,1272.65,1155.14,5228.5,5.051,1.629,5.497,117.01,4.82,283.95
4,2002-01-10,1.395,11256.07,1329.60,1656.68,10538.43,1576.437,1281.26,1156.55,5190.7,4.979,1.670,5.416,116.88,4.79,286.35
